In [3]:
_gmv_keys = [
    'temperature.output', 
    'temperature.man', 
    'temperature.interlocked', 
    'temperature.pv', 
    'temperature.sp', 
    'temperature.error', 
    'temperature.mode', 
    'agitation.output', 
    'agitation.man', 
    'agitation.interlocked', 
    'agitation.pv', 
    'agitation.sp', 
    'agitation.error', 
    'agitation.mode', 
    'ph.manUp',
    'ph.outputDown',
    'ph.error',
    'ph.pv',
    'ph.sp',
    'ph.manDown',
    'ph.outputUp',
    'ph.mode',
    'ph.interlocked',
    'do.manUp',
    'do.outputDown',
    'do.error',
    'do.pv',
    'do.sp',
    'do.manDown',
    'do.outputUp',
    'do.mode',
    'do.interlocked',
    'maingas.error',
    'maingas.man',
    'maingas.mode',
    'maingas.pv',
    'maingas.interlocked',
    'MFCs.air',
    'MFCs.n2',
    'MFCs.o2',
    'MFCs.co2',
    'condenser.output',
    'condenser.man',
    'condenser.pv',
    'condenser.sp', 
    'condenser.error', 
    'condenser.mode',
    'pressure.error',
    'pressure.pv',
    'level.error',
    'level.pv'
]

In [19]:
import threading
from hello.hello3 import open_hello, NotLoggedInError
from datetime import datetime, timedelta
import time
import sys

class _HelloLoggerTask(threading.Thread):
    def __init__(self, ip, tag, vars, poll_interval=2, start_time=None):
        
        super().__init__(daemon=True)
        # Config
        self._ip = ip
        self._vars = self._parse_vars(vars)
        self._tag = tag
        self.poll_interval = 5
        
        # Runtime
        self._data = {}
        self._h = None
        self._start_time = start_time
        
        self._iflag = threading.Event()
        self._fstop = threading.Event()
        
    def stop(self):
        self._fstop.set()
        
    def get_tagged_vars(self):
        tv = [self._tag + '.TimeStamp.Last', self._tag + '.Elapsed Time.hr']
        tv.extend(".".join((self._tag, k1, k2)) for k1, k2 in self._vars)
        return tv
        
    @property
    def tag(self):
        return self._tag
    
    def wait_for_initialized(self, timeout=10):
        self._iflag.wait(timeout)
        
    def _query(self, h, start_time, recorded, next_run):
        
        # An existential question - when did the query actually occur?
        # Did it occur as soon as the request was sent, after it was
        # recieved, or halfway between the two? The most likely scenario
        # for a delayed response is that our server software is choking 
        # for some reason, so use the timestamp immediately after return
        # as the best estimate. Otherwise, the data parsing should be fast
        # enough that the error would be measurable in microseconds. 
        
        # The loop is here to retry "infinitely" until next_run (code
        # will re-run anyway), the stop flag is set, or success. 
        # This silently swallows errors but that's fine for a proof of
        # concept of this scale. 
        
        while True:
            try:
                mv = h.gpmv()
            except Exception:
                if time.time() > next_run or self._fstop.is_set():
                    return
            else:
                break
                
        t = datetime.now()  
        data = [t, (start_time - t).total_seconds() / 3600]

        for k1, k2 in recorded:
            v = mv[k1][k2]
            data.append(v)
        self.data = data  # "Atomic"
        
    def run(self):
        
        # create some locals for brevity
        self._h = h = open_hello(self._ip)

        if self._start_time is None:
            self._start_time = datetime.now()
        start_time = self._start_time
        
        recorded = self._vars
        
        # Initial query so that self.data isn't junk
        
        self._query(h, start_time, recorded)
        self._iflag.set()
        
        while not self._fstop.is_set():
            next_run = time.time() + self.poll_interval
            self._query(h, start_time, recorded, next_run)

            left = next_run - time.time()
            if left > 0:
                self._fstop.wait(left)
        
    def _parse_vars(self, vars):
        seen = set()
        keys = []
        for k in vars:
            if k in seen:
                raise ValueError("Duplicate key found: %r"%k)
            if k not in _gmv_keys:
                raise ValueError("Invalid key: %r"%k)
            seen.add(k)
            try:
                k1, k2 = k.split(".")
            except ValueError:  # no ".", bad key
                raise ValueError("%r is an invalid key" % k) from None
            keys.append((k1, k2))
        return tuple(keys)
        
        
class HelloMultiLogger(threading.Thread):
    def __init__(self, vars, sample_interval=10):
        
        super().__init__(daemon=True)
        self._tasks = []
        self._check_vars(vars)
        self._vars = vars
        self._sample_interval = sample_interval
        self._ufuncs = []
        self._running = False
        self._fstop = threading.Event()
        
    def _check_vars(self, vars):
        for v in vars:
            if v not in _gmv_keys:
                raise ValueError("Invalid key: %s" % v)
        
    def add_callback(self, cb):
        if cb in self._ufuncs:
            return
        self._ufuncs.append(cb)
        
    def remove_callback(self, cb):
        try:
            self._ufuncs.remove(cb)
        except ValueError:
            pass
        
    def stop(self):
        for t in self._tasks:
            t.stop()
        for t in self._tasks:
            t.join()
        self._fstop.set()
        self.join()
        
    def add_logger(self, ip, tag, poll_interval=2, start=False, start_time=None):
        if self.is_alive():
            raise ValueError("Can't add tasks to already-running data logger.")
        task = _HelloLoggerTask(ip, tag, self._vars, poll_interval)
        if start:
            task.start()
        self._tasks.append(task)
        return task
    
    def start(self):
        self._running = True
        for t in self._tasks:
            if not t.is_alive():
                t.start()
        super().start()
        
    def _do_callback(self, ev, payload):
        for cb in self._ufuncs:
            try:
                cb(ev, payload)
            except Exception as e:
                sys.stderr.write("Error processing callback: %s\n" % str(e))
    
    def _send_update_cb(self, data):
        self._do_callback('ML_LOG_DATA', data)
        
    def _send_begin_cb(self):
        # the order of data sent each iteration is constant and never changes
        order = ["Timestamp", "Elapsed Time(hr)"]
        for t in self._tasks:
            order.extend(t.get_tagged_vars())
        self._do_callback("ML_BEGIN_LOG", order)
        
    def _send_end_cb(self):
        self._do_callback("ML_END_LOG", None)
        
    def run(self):
        
        start_time = datetime.now()
        for t in self._tasks:
            t.wait_for_initialized()
        
        self._send_begin_cb()
        while not self._fstop.is_set():
            
            # I went back on forth on the best way to get the data off
            # each thread when it was time to log. The format of each data
            # object is a list of tuples, and the assignment of the instance
            # attribute is atomic. Therefore, just grabbing the data object 
            # without bothering with event flags or locks is fine, assuming CPython. 
            
            ts = datetime.now()
            next_run = time.time() + self._sample_interval
            
            data = [ts, (ts-start_time).total_seconds() / 3600]
            for t in self._tasks:
                data.extend(t.data)
            self._send_update_cb(data)
            
            left = next_run - time.time()
            if left > 0:
                self._fstop.wait(left)
        self._send_end_cb()

In [20]:
def callback(event, payload):
    if event == "ML_BEGIN_LOG":
        pass
    elif event == "ML_LOG_DATA":
        pass
    elif event == "ML_END_LOG":
        pass
    else:
        raise ValueError("Unknown event: %r"%event)

def my_callback(event, payload):
    print("Got event:", event)
    if event == "ML_BEGIN_LOG":
        print(*payload)
    elif event == "ML_LOG_DATA":
        print(*payload)
    elif event == "ML_END_LOG":
        pass
    else:
        raise ValueError("Unknown event: %r"%event)
        
def csv_writer(filename):
    f = None
    def csv_writer_cb(event, payload):
        nonlocal f
        if event == "ML_BEGIN_LOG":
            f = open(filename, 'w')
            f.write(",".join(map(str, payload)))
            f.write("\n")
        elif event == "ML_LOG_DATA":
            f.write(",".join(map(str, payload)))
            f.write("\n")
        elif event == "ML_END_LOG":
            f.close()
        else:
            raise ValueError("Unknown event: %r"%event)
    
    return csv_writer_cb

In [21]:
vars_i_care_about = [
    'agitation.pv',
    'temperature.pv'
]

ml = HelloMultiLogger(vars_i_care_about, 3)

ml.add_logger("192.168.1.14", "R&D1", 1)
ml.add_logger("192.168.1.16", "R&D2", 1)

ml.add_callback(my_callback)
ml.add_callback(csv_writer("test_multilogger.csv"))

ml.start()

Got event: ML_BEGIN_LOG
Timestamp Elapsed Time(hr) R&D1.TimeStamp.Last R&D1.Elapsed Time.hr R&D1.agitation.pv R&D1.temperature.pv R&D2.TimeStamp.Last R&D2.Elapsed Time.hr R&D2.agitation.pv R&D2.temperature.pv
Got event: ML_LOG_DATA
2018-10-01 16:53:28.200363 0.0002878958333333333 2018-10-01 16:53:28.187398 -0.0002856797222222222 0 27.067668914794922 2018-10-01 16:53:27.982377 -0.00022817555555555558 0 31.601276397705078
Got event: ML_LOG_DATA
2018-10-01 16:53:31.200514 0.001121271111111111 2018-10-01 16:53:28.380205 -0.0003392372222222222 0 27.067668914794922 2018-10-01 16:53:27.982377 -0.00022817555555555558 0 31.601276397705078
Got event: ML_LOG_DATA
2018-10-01 16:53:34.202049 0.0019550308333333333 2018-10-01 16:53:33.353377 -0.001720673888888889 0 27.067981719970703 2018-10-01 16:53:32.995425 -0.0016206888888888888 0 31.605201721191406
Got event: ML_LOG_DATA
2018-10-01 16:53:37.203109 0.002788658611111111 2018-10-01 16:53:33.353377 -0.001720673888888889 0 27.067981719970703 2018-10-

In [22]:
ml.stop()

Got event: ML_END_LOG
